# Requisito 2 — Análise das Redes de Coautoria (PPgEEC)

Este notebook apresenta uma análise das redes de coautoria do PPgEEC ao longo dos períodos de avaliação: 2010–2012, 2013–2016, 2017–2020 e 2021–2024. As visualizações foram construídas com base nas especificações do Requisito 2, que define que o tamanho dos vértices seja proporcional ao número de vizinhos (grau), as arestas vermelhas representem colaborações entre membros permanentes do programa, e a largura das arestas seja determinada pela quantidade de citações compartilhadas. Os cinco autores mais conectados foram destacados em amarelo com rótulo. Todas as redes foram filtradas para exibir apenas a maior componente conectada, e renderizadas com layout de força para facilitar a interpretação visual.


In [ ]:
# Importação das bibliotecas necessárias
import networkx as nx
import matplotlib.pyplot as plt

# Função principal de visualização de uma rede
# Recebe um grafo, um título para a imagem e um parâmetro opcional para aplicar coloração por comunidades

def visualizar_rede_final(grafo, titulo, aplicar_comunidades=False):
    # Seleciona a maior componente conectada do grafo
    componentes = sorted(nx.connected_components(grafo), key=len, reverse=True)
    grafo = grafo.subgraph(componentes[0]).copy()

    # Calcula o grau de cada vértice e identifica os 5 mais conectados
    graus = dict(grafo.degree())
    maiores = sorted(graus.items(), key=lambda x: x[1], reverse=True)[:5]
    top5 = [n for n, _ in maiores]

    # Gera o layout com espaçamento controlado (k)
    pos = nx.spring_layout(grafo, seed=42, k=3)

    # Define tamanho dos nós com base no grau
    tamanhos = [graus[n] * 20 + 30 for n in grafo.nodes()]

    # Prepara atributos das arestas: cor, largura e opacidade
    cores_arestas = []
    larguras_arestas = []
    alphas_arestas = []
    for u, v in grafo.edges():
        is_perm_u = grafo.nodes[u].get('is_permanent', False)
        is_perm_v = grafo.nodes[v].get('is_permanent', False)
        cor = 'red' if is_perm_u and is_perm_v else 'black'
        cores_arestas.append(cor)
        larguras_arestas.append(0.5 + grafo[u][v].get('citation_num', 1) * 0.05)
        alphas_arestas.append(1.0 if cor == 'red' else 0.05)

    # Aplica coloração por comunidades
    if aplicar_comunidades:
        from networkx.algorithms.community import greedy_modularity_communities
        import matplotlib.cm as cm
        comunidades = list(greedy_modularity_communities(grafo))
        cores_comunidades = {n: i for i, c in enumerate(comunidades) for n in c}
        cores_nos = [cores_comunidades[n] for n in grafo.nodes()]
        cmap = cm.get_cmap('Set3')
    else:
        cores_nos = 'skyblue'
        cmap = None

    # Gera a visualização do grafo
    plt.figure(figsize=(16, 12), facecolor='white')
    nx.draw_networkx_nodes(
        grafo, pos, node_size=tamanhos, node_color=cores_nos,
        cmap=cmap, edgecolors='gray', linewidths=0.5
    )

    for i, (u, v) in enumerate(grafo.edges()):
        nx.draw_networkx_edges(
            grafo, pos, edgelist=[(u, v)],
            edge_color=cores_arestas[i], width=larguras_arestas[i],
            alpha=alphas_arestas[i]
        )

    # Destaque dos top 5 vértices
    nx.draw_networkx_nodes(
        grafo, pos, nodelist=top5, node_color='gold',
        node_size=500, edgecolors='black', linewidths=2
    )
    nx.draw_networkx_labels(
        grafo, pos, labels={n: n for n in top5},
        font_size=18, font_color='black', font_weight='bold'
    )

    # Finaliza e salva a figura
    plt.title(titulo, fontsize=16)
    plt.axis('off')
    nome_arquivo = f"{titulo.replace(' ', '_')}.png"
    plt.savefig(nome_arquivo, dpi=300, facecolor='white')
    plt.show()
    print(f"✅ Imagem salva como: {nome_arquivo}")

In [ ]:
# Dicionário com caminhos e títulos
periodos = {
    "2010-2012.gexf": "Rede de Coautoria 2010-2012",
    "2013-2016.gexf": "Rede de Coautoria 2013-2016",
    "2017-2020.gexf": "Rede de Coautoria 2017-2020",
    "2021-2024.gexf": "Rede de Coautoria 2021-2024"
}

# Loop para carregar e gerar imagens
for caminho, titulo in periodos.items():
    G = nx.read_gexf(caminho)
    print(f"\n📊 Visualizando: {titulo}")
    visualizar_rede_final(G, titulo)

Durante o período de 2010 a 2012, observa-se uma rede relativamente centralizada, com poucos autores ocupando posições de alta conectividade. As colaborações entre membros permanentes são marcadas por arestas vermelhas e já indicam um núcleo bem formado.

Entre 2013 e 2016, a rede se expande significativamente. Novos nós passam a compor o centro da rede e surgem novas conexões entre permanentes, agora com maior frequência e intensidade. Os top 5 autores mantêm posições centrais, embora alguns nomes mudem em relação ao triênio anterior.

Na análise de 2017 a 2020, a densidade da rede é visivelmente maior. A estrutura se torna mais complexa, e é possível identificar agrupamentos colaborativos com maior clareza. Os autores mais influentes continuam ocupando o centro da rede, conectando vários grupos e mantendo coesão entre diferentes partes da estrutura.

Por fim, no período mais recente (2021 a 2024), nota-se uma maior dispersão e diversidade na rede. Muitos novos autores aparecem nas bordas, e a colaboração entre permanentes permanece presente, mas não é tão predominante quanto antes. Essa dinâmica sugere a entrada de novos participantes no programa e uma distribuição mais heterogênea das colaborações.

No geral, observa-se uma evolução consistente da rede, tanto em tamanho quanto em complexidade estrutural. O crescimento em número de nós e arestas reflete o amadurecimento do programa e o aumento nas colaborações. A presença de autores centrais recorrentes ao longo dos períodos também é um indício de liderança ou maior engajamento em atividades de pesquisa colaborativa. A forte presença de arestas vermelhas no período intermediário demonstra que os membros permanentes atuaram como eixos de integração e produção durante essa fase.

Apesar dos bons resultados, algumas dificuldades foram encontradas. O layout inicial das redes se apresentava muito denso, prejudicando a interpretação visual. Foram realizados ajustes nos parâmetros de espaçamento (k), nos tamanhos dos nós, na opacidade das arestas e na identificação dos autores mais conectados para melhorar a clareza. Além disso, como os nós não estão nomeados com nomes reais, mas com IDs numéricos, a identificação direta dos autores ficou limitada.

Pode-se hipotetizar que os autores mais centrais estão ligados à coordenação de linhas de pesquisa, projetos interinstitucionais ou possuem longa trajetória dentro do programa. A evolução da rede ao longo do tempo sugere um programa mais inclusivo, com maior interação externa e crescimento colaborativo.